In [ ]:
import boto3
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO

In [ ]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [ ]:
from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

### Resumo da análise

- Existem colunas duplicadas que representam a mesma variável [``start_season``, ``start_of_season``], além de [``end_season``, ``end_of_season``], [``peak_of_season``, ``peak_season``]
- Existem dados duplicados considerando o ``eopatch_location``
- Não existe mais a coluna ``id_talhao``?

### Global Variables

In [ ]:
GEOPACKAGE_PATH = ["/agrilearn_app/datasets/teste_pre_safra_2024_2025/geopackage/raw/data_merged_crop_120_06_08_2024_input_mvp.gpkg"]

label_monitoring_class = 'gt_class'
label_eopatch_path = "sentinel_eopatch_current" # 'sentinel_eopatch_current' #eopath_location

# 1. Read Datasets

### Data description
- monitoring_class: é o ``conjunto`` que representa a cultura de interesse do cliente ou da amostra de treinamento (SOYBEAN, CORN)
- period: é o ``texto`` que representa a safra agricola (2023/2024, 2024/2025)
- fonte: é o ``texto`` da origem dos dados (mapas temáticos, banco de dados)
- state: é o ``texto`` do estado do polígono
- area: é o ``ponto flutuante`` da área em métros
- micro: é o ``inteiro`` que representa a micro região
- start_season: é o ``timestamp`` que inicia a safra pra determinada cultura (monitoring_class) e safra (period)
- end_season: é o  ``timestamp`` que termina a safra pra determinada cultura (monitoring_class) e safra (period)
- peak_start: é o ``timestamp`` que inicia o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period)
- peak_end: é o ``timestamp`` que termina o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period) (Obs: temos um calendário pra isso, um arquivo yaml)
- eopath_location: é o ``texto`` que representa o caminho onde o eopatch está salvo (imagens p/ inferência)
- start_of_season: é o ``timestamp`` da data da emergencia da cultura
- peak_of_season: é o ``timestamp`` da data do pico vegetativo da cultura
- end_of_season: é o ``timestamp`` da data da colheita da cultura
- length_of_season: é o ``inteiro`` com a duração do cultivo em dias
- is_valid_metrics:
- sos_valid:
- pos_valid:
- eos_valid:
- los_valid:
- planting_start: é o ``timestamp`` da data do inicio do plantio.
- planting_end: é o ``timestamp`` da data do fim do plantio.
- start_of_cycle: é o ``timestamp`` da data do inio do cultivo segundo o calendário agrícola
- end_of_cycle: é o ``timestamp`` da data do fim do caledário de cultivo segundo calendário agrícola
- length_of_cycle: é o ``inteiro`` que representa o tamanho do ciclo em dias
- is_valid:
- is_valid_POS:
- is_valid_LOS:
- set_type: é o ``conjunto`` que representar a divisão do dado entre treino, validação e teste
- sampled_date: é o ``texto`` com a data do sample em mês e ano
- cultura_2:
- geometry: é a ``geometria`` do polígono 

In [ ]:
gdfs = []

# Lê cada Geopackage e adiciona o GeoDataFrame à lista
for path in GEOPACKAGE_PATH:
    gdf = gpd.read_file(path)
    gdf['dataset_source'] = path  # Adiciona a coluna de origem
    gdfs.append(gdf)

# Concatena todos os GeoDataFrames em um único GeoDataFrame
if gdfs:
    df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    print(f"Dataset final possui {df.shape[0]} linhas")
    df.head()
else:
    print("Nenhum GeoDataFrame válido encontrado.")

In [ ]:
df = df[~df['set'].isin(['run_2082_mvp_5972_bb_areas_crop_120.gpkg', 'run_2082_mvp_344_teste_concordancia_crop_120.gpkg'])]

In [ ]:
len(df.columns)

In [ ]:
df.dropna(axis=1, how='all', inplace=True)

In [ ]:
len(df.columns)

In [ ]:
datetime_columns = ['start_season', 
                    'end_season', 
                    'peak_start', 
                    'peak_end',
                    'start_of_season',
                    'peak_of_season',
                    'end_of_season',
                    'planting_start',
                    'planting_end',
                    'start_of_cycle',
                    'end_of_cycle'                  
                   ]

    # Verifica se a coluna existe, se existir formata, senão printa mensagem
for col in datetime_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
        except Exception as e:
            print(f"Erro ao formatar a coluna {col}: {e}")
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")
        # Exibe informações das colunas datetime

In [ ]:
df[label_monitoring_class].value_counts()

In [ ]:
df['state'].value_counts()

In [ ]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

In [ ]:
print(f"Dados de {df['start_season'].min()} a {df['end_season'].max()}")

In [ ]:
list(df.columns)

# 02. Data Integration in columns

In [ ]:
df[['gt_class', 'monitoring_class']].sample(10)

In [ ]:
df[['eopath_location', 'sentinel_eopatch_current']].sample(10)

In [ ]:
df['monitoring_class'] = df['gt_class']

# 03. Check Data Quality

## 3.1 Check NaN Values

In [ ]:
# Calcula a quantidade de registros nulos e a porcentagem de registros nulos
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)

In [ ]:
df_check_NaN.iloc[:30]

## 3.2 Drop Columns

In [ ]:
# columns_to_delete = ['LOS',
#                      'start_of_season',
#                      'end_of_season',
#                      'start_of_season', 
#                     ]
# df.drop(columns=columns_to_delete, inplace=True)

- Q1) Quais colunas são geradas por nós e quais são coletadas? 
- Q2) Qual a razão dos dados nulos?
- Q3) Qual o melhor conjunto de colunas eu devo usar para considerar dados duplicados uma vez que o field_id não está disponível para alguns datasets?

In [ ]:
df_check_duplicated = df[df.duplicated(subset=['geometry', 'period'], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby(['geometry', 'period']).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

In [ ]:
# df_check_duplicated.to_csv('/agrilearn_app/datasets/teste_pre_safra_2024_2025/reports/report_duplicated_values_geometry_period_eopatch_location_bb_areas_and_teste_concordancia_SOYBEAN_5916_CORN_180_COTTON_51_PASTURE_50_SUGAR_CANE_50_WHEAT_50_RICE_19.csv',
#                            sep=';',
#                            decimal=',',
#                            float_format='%.4f')

In [ ]:
shape_before = df.shape[0]
df.drop_duplicates(subset=['geometry', 'period'], inplace=True)
print(f"Removed records: {shape_before - df.shape[0]}, Percentage: {(shape_before - df.shape[0]) / shape_before * 100:.2f}%")

# 04. Univariate Data Analysis

### 4.1 Analysing Target y ``monitoring_class``

In [ ]:
df_check_NaN.loc['monitoring_class']

In [ ]:
df['monitoring_class'].nunique()

In [ ]:
pd.concat([df['monitoring_class'].value_counts(),
          df['monitoring_class'].value_counts(normalize=True)*100], axis=1)

### 4.2 Analysing dados por ``estado``

In [ ]:
df_check_NaN.loc['state']

In [ ]:
df['state'].nunique()

In [ ]:
pd.concat([df['state'].value_counts(),
          df['state'].value_counts(normalize=True)*100], axis=1)

### 4.3 Analysis ``period``

In [ ]:
df_check_NaN.loc['period']

In [ ]:
df['period'].nunique()

In [ ]:
pd.concat([df['period'].value_counts(),
          df['period'].value_counts(normalize=True)*100], axis=1)

### 4.7 Analysis of ``length_of_season (LOS)``

In [ ]:
df_check_NaN.loc['length_of_season']

In [ ]:
df['length_of_season'] = df['length_of_season'].astype(float)

In [ ]:
df['length_of_season'].describe()

In [ ]:
pd.concat([df['length_of_season'].value_counts(),
          df['length_of_season'].value_counts(normalize=True)*100], axis=1)

In [ ]:
df[df['length_of_season'] < 100]

### 4.8 Analysis of ``area``

In [ ]:
df_check_NaN.loc['area']

In [ ]:
pd.concat([df['area'].value_counts(),
          df['area'].value_counts(normalize=True)*100], axis=1)

### 4.10 Analysis of ``cultura_2``

In [ ]:
# df_check_NaN.loc['cultura_2']

In [ ]:
# pd.concat([df['cultura_2'].value_counts(),
#           df['cultura_2'].value_counts(normalize=True)*100], axis=1)

### 4.11 Analysis of ``fonte``

In [ ]:
df_check_NaN.loc['fonte']

In [ ]:
pd.concat([df['fonte'].value_counts(),
          df['fonte'].value_counts(normalize=True)*100], axis=1)

### 4.12 Analysis of ``eopath_location``

In [ ]:
df_check_NaN.loc[label_eopatch_path]

In [ ]:
df.dropna(subset=[label_eopatch_path])

In [ ]:
df[label_eopatch_path].nunique()

In [ ]:
df.shape[0]

In [ ]:
df_check_duplicated = df[df.duplicated(subset=[label_eopatch_path], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby([label_eopatch_path]).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

In [ ]:
# Verifica se há duplicatas na coluna especificada
duplicates = df['eopath_location'].duplicated(keep=False)

# Calcula a quantidade de dados duplicados e o percentual em relação ao dataset final
total_rows = len(df)
duplicate_count = duplicates.sum()
duplicate_percentage = (duplicate_count / total_rows) * 100

print(f"Quantidade de dados duplicados: {duplicate_count}")
print(f"Percentual de dados duplicados: {duplicate_percentage:.2f}%")

* **Question**: O mesmo geopackage está apontando par ao mesmo eopatch_location? *

### 4.13 Check ``start_season`` and ``end_season``

In [ ]:
df_check_NaN.loc[['start_season', 'end_season']]

In [ ]:
df[['start_season', 'end_season']].describe()

In [ ]:
df.groupby(df['start_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['end_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['start_season'] == df['end_season']]

### 4.14 Check ``start_season`` and ``end_season``

In [ ]:
df_check_NaN.loc[['peak_start', 'peak_end']]

In [ ]:
df[['peak_start', 'peak_end']].describe()

In [ ]:
df[['peak_start', 'peak_end']].describe()

In [ ]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['peak_end'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['peak_start'] == df['peak_end']]

In [ ]:
df['peak_start'].dt.year.unique()

In [ ]:
df['peak_end'].dt.year.unique()

In [ ]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

### 4.15 Check ``planting_start`` and ``planting_end``

In [ ]:
df_check_NaN.loc[['planting_start', 'planting_end']]

In [ ]:
df[['planting_start', 'planting_end']].describe()

In [ ]:
df.groupby(df['planting_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['planting_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['planting_start'] == df['planting_end']]

In [ ]:
df['planting_start'].dt.year.unique()

In [ ]:
df['planting_end'].dt.year.unique()

# 5. Multivariate Analysis

In [ ]:
df[label_monitoring_class].value_counts()

In [ ]:
columns_to_group = ['period', 'state', label_monitoring_class]
report_distribuition = df.groupby(columns_to_group).agg(count=(label_eopatch_path, 'count')).reset_index()

# Calcula o percentual em relação ao total
total_count = len(df)
report_distribuition['percentage'] = (report_distribuition['count'] / total_count) * 100

In [ ]:
report_distribuition

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in df[label_monitoring_class].value_counts().items()])
result_string

In [ ]:
report_distribuition.to_csv(f'/agrilearn_app/datasets/base/reports/check_distribuition_by_class_and_state_{result_string}.csv',
    sep=';',
    decimal=',',
    float_format='%.2f',
)

In [ ]:
df.to_file(f"/agrilearn_app/datasets/base/geopackage/processed/bb_areas_and_teste_concordancia_{result_string}.gpkg", driver='GPKG', engine='fiona')